**TODO :**
- Recheck Augmentations
- Recheck LAB normalization
- sampler for faster convergence ?

In [9]:
# %load_ext nb_black
%load_ext autoreload
%autoreload 2

### Initialization

### Imports

In [10]:
import os
import sys
import torch
import warnings
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

sys.path.append("../code/")
# os.environ['CUDA_VISIBLE_DEVICES'] = "1,0"
warnings.simplefilter("ignore", UserWarning)

In [11]:
extra = pd.read_csv('../input/external_data/theos_tiff/train_extra.csv')

In [13]:
extra['Unnamed: 0']

0        SESCAM_1_0
1        SESCAM_102
2     SAS_21904_001
3     SAS_21908_001
4        VUHSK_1502
5        SESCAM_7_0
6        VUHSK_2072
7        SESCAM_9_0
8     SAS_21937_001
9     SAS_21883_001
10    SAS_21891_001
11       VUHSK_1912
12       SESCAM_5_0
13       SESCAM_2_0
14       VUHSK_1992
15    SAS_21930_001
16       VUHSK_1622
17       VUHSK_1352
18    SAS_21942_001
19       SESCAM_3_0
20       VUHSK_1272
21       VUHSK_1762
22       VUHSK_1702
23    SAS_21896_001
24       SESCAM_6_0
25       SESCAM_4_0
26       SESCAM_8_0
27    SAS_21915_001
28       VUHSK_1832
29    SAS_21924_001
30       VUHSK_1432
Name: Unnamed: 0, dtype: object

In [3]:
from training.main import k_fold

from utils.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
    update_overall_logs,
)

from params import *

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### Load

In [4]:
df_info = pd.read_csv(DATA_PATH + f"HuBMAP-20-dataset_information.csv")
df_mask = pd.read_csv(DATA_PATH + "train.csv")

## Training

In [5]:
BATCH_SIZES = {
    "resnet18": 64,
    "resnet34": 32,
    "resnext50_32x4d": 32,
    "se_resnext50_32x4d": 32,
    "efficientnet-b0": 32,
    "efficientnet-b1": 32,
    "efficientnet-b2": 32,
    "efficientnet-b3": 16,
    "efficientnet-b4": 16,
    "efficientnet-b5": 16,
    "efficientnet-b6": 8,
}

In [6]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1

    device = "cuda" if torch.cuda.is_available() else "cpu"
    save_weights = True
    sampling_mode = 'centered' # chose between 'convhull', 'centered', 'random', 'visible'
    
    # Images
    tile_size = 256
    reduce_factor = 4
    on_spot_sampling = 0.99
    overlap_factor = 1.5

    img_dir = DATA_PATH + f"train_{tile_size}_red_{reduce_factor}"
    mask_dir = DATA_PATH + f"masks_{tile_size}_red_{reduce_factor}"

    # k-fold
    cv_column = "5fold"
    random_state = 0
    selected_folds = [0, 1, 2, 3, 4]

    # Model
    encoder = "efficientnet-b2"  # "resnet18" "resnext50_32x4d", "resnet34", "efficientnet-b5"
    decoder = "Unet"  # "Unet", "DeepLabV3Plus"
    use_bot = False
    use_fpn = False
    double_model = False
    encoder_weights = "imagenet"
    num_classes = 1

    # Training
    loss = "BCEWithLogitsLoss"  # "SoftDiceLoss" / "BCEWithLogitsLoss"  / "lovasz"
    activation = "none" if loss == "lovasz" else "sigmoid"

    optimizer = "Adam"
    
    batch_size = BATCH_SIZES[encoder]
    
    if tile_size == 512:
        batch_size = batch_size // 2
        
    if batch_size >= 32:
        epochs = 50
    elif batch_size >= 16:
        epochs = 40
    elif batch_size >= 6:
        epochs = 30
    else:
        epochs = 25
    
    iter_per_epoch = 5000
    lr = 1e-3
    swa_first_epoch = 50

    warmup_prop = 0.05
    val_bs = batch_size * 2

    first_epoch_eval = 0
    
    use_fp16 = True
    # external features : https://www.kaggle.com/baesiann/glomeruli-hubmap-external-1024x1024
    # download this and put folders in input/external_data
    # api command for download (requires kaggle api + credentials):
    # kaggle datasets download -d baesiann/glomeruli-hubmap-external-1024x1024
    use_external = 0.2 # percentage of sampling on external data (training only), None to disable
    
    #pseudo labels
    # download test set & ext pseudo labels : https://www.kaggle.com/theoviel/soft-pl-932
    # save in input/external_data/PL/
    # api command :  kaggle datasets download -d theoviel/soft-pl-932   
    # external data from : https://www.kaggle.com/iafoss/hubmap-ex
    # api command : kaggle datasets download -d iafoss/hubmap-ex
    # save to input/test/
    use_pseudo_label = 0.2 # percentage adding to use_external, None to disable
    soft_labels = False

In [7]:
DEBUG = False
log_folder = None

In [8]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")

metrics = k_fold(Config, log_folder=log_folder)

Logging results to ../logs/2021-04-28/2/
Creating in-memory dataset ...
Done in 129 seconds.

-------------   Fold 1 / 5  -------------

    -> 10046291 trainable parameters
    -> Validation images : ['2f6ecfcdf', 'b2dc8411c', '4ef6695ce'] 

Epoch 01/50 	 lr=4.0e-04	 t=80s	loss=0.380	val_loss=0.112 	 dice=0.7240
Epoch 02/50 	 lr=8.0e-04	 t=80s	loss=0.075	val_loss=0.035 	 dice=0.5178
Epoch 03/50 	 lr=9.9e-04	 t=81s	loss=0.041	val_loss=0.018 	 dice=0.8985
Epoch 04/50 	 lr=9.7e-04	 t=77s	loss=0.036	val_loss=0.010 	 dice=0.9373
Epoch 05/50 	 lr=9.5e-04	 t=82s	loss=0.029	val_loss=0.009 	 dice=0.9461
Epoch 06/50 	 lr=9.3e-04	 t=80s	loss=0.028	val_loss=0.008 	 dice=0.9462
Epoch 07/50 	 lr=9.1e-04	 t=81s	loss=0.026	val_loss=0.009 	 dice=0.9303
Epoch 08/50 	 lr=8.8e-04	 t=79s	loss=0.027	val_loss=0.030 	 dice=0.8701
Epoch 09/50 	 lr=8.6e-04	 t=79s	loss=0.026	val_loss=0.008 	 dice=0.9518
Epoch 10/50 	 lr=8.4e-04	 t=81s	loss=0.025	val_loss=0.008 	 dice=0.9477
Epoch 11/50 	 lr=8.2e-04	 t=79s	loss=


    -> 10046291 trainable parameters
    -> Validation images : ['aaa6a05cc', 'e79de561c', 'c68fe75ea'] 

Epoch 01/50 	 lr=4.0e-04	 t=80s	loss=0.381	val_loss=0.117 	 dice=0.6535
Epoch 02/50 	 lr=8.0e-04	 t=81s	loss=0.075	val_loss=0.034 	 dice=0.8970
Epoch 03/50 	 lr=9.9e-04	 t=83s	loss=0.040	val_loss=0.021 	 dice=0.8517
Epoch 04/50 	 lr=9.7e-04	 t=78s	loss=0.033	val_loss=0.018 	 dice=0.8990
Epoch 05/50 	 lr=9.5e-04	 t=81s	loss=0.030	val_loss=0.017 	 dice=0.8935
Epoch 06/50 	 lr=9.3e-04	 t=80s	loss=0.028	val_loss=0.015 	 dice=0.8964
Epoch 07/50 	 lr=9.1e-04	 t=81s	loss=0.028	val_loss=0.018 	 dice=0.8280
Epoch 08/50 	 lr=8.8e-04	 t=78s	loss=0.026	val_loss=0.016 	 dice=0.8927
Epoch 09/50 	 lr=8.6e-04	 t=80s	loss=0.026	val_loss=0.015 	 dice=0.8777
Epoch 10/50 	 lr=8.4e-04	 t=80s	loss=0.025	val_loss=0.016 	 dice=0.8731
Epoch 11/50 	 lr=8.2e-04	 t=80s	loss=0.025	val_loss=0.016 	 dice=0.8941
Epoch 12/50 	 lr=8.0e-04	 t=80s	loss=0.023	val_loss=0.015 	 dice=0.8796
Epoch 13/50 	 lr=7.8e-04	 t=8